In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import ArxivLoader
from langchain.vectorstores import Chroma
from langgraph.graph import StateGraph, END

In [2]:
llm = Ollama(model="gemma:2b")  # for summaries, comparisons
embeddings = OllamaEmbeddings(model="nomic-embed-text")  # fast embeddings

C:\Users\rathi\AppData\Local\Temp\ipykernel_20340\2320363549.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma:2b")  # for summaries, comparisons
C:\Users\rathi\AppData\Local\Temp\ipykernel_20340\2320363549.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")  # fast embeddings


In [3]:
query = "transformer models in healthcare"
loader = ArxivLoader(query=query, max_docs=2)
docs = loader.load()

In [4]:
db = Chroma.from_documents(docs, embeddings, persist_directory="chroma_db")
retriever = db.as_retriever(search_kwargs={"k": 3})

In [5]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# -------------------------
# 1. Define State Schema
# -------------------------
class GraphState(TypedDict, total=False):
    query: str
    papers: List[str]
    summaries: List[str]
    comparison: str
    literature_review: str

# -------------------------
# 2. Define Agents
# -------------------------
def retriever_agent(state: GraphState) -> GraphState:
    query = state["query"]
    results = retriever.get_relevant_documents(query)
    return {"papers": results}

def summarizer_agent(state: GraphState) -> GraphState:
    summaries = []
    for doc in state["papers"]:
        prompt = f"Summarize this paper in 4 sections: Background, Methods, Results, Conclusion.\n\n{doc.page_content[:2000]}"
        summary = llm.invoke(prompt)
        summaries.append(summary)
    return {"summaries": summaries}

def comparator_agent(state: GraphState) -> GraphState:
    prompt = f"Compare these research summaries and highlight similarities & differences:\n\n{state['summaries']}"
    comparison = llm.invoke(prompt)
    return {"comparison": comparison}

def synthesizer_agent(state: GraphState) -> GraphState:
    prompt = f"Write a short literature review draft based on the following comparison:\n\n{state['comparison']}"
    lit_review = llm.invoke(prompt)
    return {"literature_review": lit_review}

# -------------------------
# 3. Build Graph
# -------------------------
graph = StateGraph(GraphState)

graph.add_node("retriever", retriever_agent)
graph.add_node("summarizer", summarizer_agent)
graph.add_node("comparator", comparator_agent)
graph.add_node("synthesizer", synthesizer_agent)

graph.add_edge("retriever", "summarizer")
graph.add_edge("summarizer", "comparator")
graph.add_edge("comparator", "synthesizer")
graph.add_edge("synthesizer", END)

graph.set_entry_point("retriever")

app = graph.compile()

# -------------------------
# 4. Run Workflow
# -------------------------
output = app.invoke({"query": "transformer models in healthcare"})
print("\n📑 Literature Review Draft:\n")
print(output["literature_review"])


C:\Users\rathi\AppData\Local\Temp\ipykernel_20340\1424376213.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



📑 Literature Review Draft:

## Literature Review Draft

**Review of Comparison of Research Summaries**

The two research summaries offer valuable insights into the strategic importance of data governance, strategic competitive advantages, and the transformative potential of healthcare-specific foundation models. Despite their differing focuses, both summaries highlight the challenges faced by healthcare organizations in managing fragmented data structures, lack of data governance frameworks, and stringent data privacy regulations.

While the first summary provides a general overview of the strategic imperative for healthcare organizations to develop proprietary foundation models, the second summary focuses on the application of transformer-based models in intra-operative anesthesia management data. This difference in scope enables the second summary to delve into the specific performance of these models, offering valuable insights into their potential benefits and challenges.

Further